<a href="https://colab.research.google.com/github/YoheiFukuhara/recommender-system/blob/main/02_preprocess_shrink.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from collections import Counter

import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
BASE_PATH = '/content/drive/MyDrive/ColabNotebooks/ML/Recommend/output/'
df = pd.read_csv(BASE_PATH+'edited_rating.csv')

In [ ]:
pd.options.display.float_format = '{:.2f}'.format
df.info()
display(df.describe())
display(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   movie_idx  int64  
dtypes: float64(1), int64(3)
memory usage: 610.4 MB


,userId,movieId,rating,movie_idx
count,20000263.00,20000263.00,20000263.00,20000263.00
mean,69044.87,9041.57,3.53,3612.60
std,40038.63,19789.48,1.05,4154.44
min,0.00,1.00,0.50,0.00
25%,34394.00,902.00,3.00,887.00
50%,69140.00,2167.00,3.50,2085.00
75%,103636.00,4770.00,4.00,4677.00
max,138492.00,131262.00,5.00,26743.00


,userId,movieId,rating,movie_idx
0,0,2,3.50,2
1,0,29,3.50,29
2,0,32,3.50,32
3,0,47,3.50,47
4,0,50,3.50,50
...,...,...,...,...
20000258,138492,68954,4.50,13821
20000259,138492,69526,4.50,13929
20000260,138492,69644,3.00,13942
20000261,138492,70286,5.00,14060


In [ ]:
print(f'unique number of user id: {df.userId.nunique()}')
print(f'unique number of movie id: {df.movie_idx.nunique()}')

unique number of user id: 138493
unique number of movie id: 26744


In [ ]:
%%time
# number of users and movies to keep
user_ids = [id for id, _ in Counter(df.userId).most_common(10000)]
movie_ids = [id for id, _ in Counter(df.movie_idx).most_common(2000)]

CPU times: user 8.4 s, sys: 28.8 ms, total: 8.43 s
Wall time: 8.42 s


In [ ]:
%%time
df_small = df[df.userId.isin(user_ids) & df.movie_idx.isin(movie_ids)].copy()
df_small.info()
display(df_small.describe())
display(df_small)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5392025 entries, 960 to 19998296
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   movie_idx  int64  
dtypes: float64(1), int64(3)
memory usage: 205.7 MB


,userId,movieId,rating,movie_idx
count,5392025.00,5392025.00,5392025.00,5392025.00
mean,68832.46,6735.17,3.45,3265.44
std,40022.11,14985.54,1.02,3363.07
min,10.00,1.00,0.50,1.00
25%,34312.00,1172.00,3.00,1151.00
50%,69345.00,2289.00,3.50,2206.00
75%,103165.00,4033.00,4.00,3941.00
max,138473.00,109374.00,5.00,22831.00


,userId,movieId,rating,movie_idx
960,10,1,4.50,1
961,10,10,2.50,10
962,10,19,3.50,19
963,10,32,5.00,32
964,10,39,4.50,39
...,...,...,...,...
19998291,138473,4993,5.00,4900
19998292,138473,5349,3.00,5255
19998293,138473,5378,4.00,5284
19998295,138473,5449,4.00,5355


CPU times: user 4.14 s, sys: 20.4 ms, total: 4.16 s
Wall time: 4.16 s


In [ ]:
def create_old_map(ids):
    id_map = {}
    for i, old in enumerate(ids):
        id_map[old] = i
    print(len(id_map))
    return id_map

new_user_id_map = create_old_map(user_ids)
new_movie_id_map = create_old_map(movie_ids)

10000
2000


In [ ]:
%%time
df_small.loc[:,'userId'] = df_small.userId.map(lambda x: new_user_id_map[x])
df_small.loc[:,'movie_idx'] = df_small.movie_idx.map(lambda x: new_movie_id_map[x])

CPU times: user 4.71 s, sys: 286 ms, total: 5 s
Wall time: 4.97 s


In [ ]:
display(df_small.describe())

,userId,movieId,rating,movie_idx
count,5392025.00,5392025.00,5392025.00,5392025.00
mean,4064.70,6735.17,3.45,686.57
std,2904.37,14985.54,1.02,549.58
min,0.00,1.00,0.50,0.00
25%,1471.00,1172.00,3.00,211.00
50%,3614.00,2289.00,3.50,552.00
75%,6432.00,4033.00,4.00,1078.00
max,9999.00,109374.00,5.00,1999.00


In [ ]:
df_small.to_csv(BASE_PATH+'very_small_rating.csv', index=False)